# 0. Intro

# 1.


In [97]:
P = 0.5         # oversample数据的比例
N_cluster = 3   # cluster 类型
RESCALE = True  # 是否需要重新整理粒度？


# Target data
target_path = r"../data/targets"
arr_path_root = r"../data/arr_selected"

if RESCALE:
    SCALE = 5   # 粒度整理为5
    col_names_2 = ['N','P'] # 保留列名
    target_path_DA = r"../data/targets_"+str(SCALE)+"_DA_P="+str(P)+r"_N_c="+str(N_cluster) # Target data after DA

else:
    target_path_DA = r"../data/targets_DA_P="+str(P)+r"_N_c="+str(N_cluster)
print(target_path_DA)


../data/targets_5_DA_P=0.5_N_c=3


In [87]:
import os
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
from numpy import random
import collections
from scipy.spatial.distance import cdist
from tqdm import tqdm

In [88]:

# all_path里有全部的data地址作为list
target_all_path = os.listdir(target_path)

# 2. DA
1. 根据RESCALE参数决定要不要整理粒度

In [75]:
Total_data_records = 0

In [76]:
for i in tqdm(range(len(target_all_path))):

    # Read in the data
    data_path = os.path.join(target_path,target_all_path[i])
    data = pd.read_csv(data_path,encoding="utf-8")
    data = np.array(data.iloc[:,0]).reshape(-1, 1)

    ##################### KMeans and compute dist and prob  #####################
    # Do the KMeans
    n_cluster = min(N_cluster,len(np.unique(data))) # 有可能data的nunique还没有N_cluster大

    kmeans = KMeans(n_clusters=n_cluster, random_state=0).fit(data)
    centers = np.array(kmeans.cluster_centers_, dtype=int)  # 返回center
    labels =  pd.DataFrame(kmeans.labels_)                  # 返回每一点的cluster标签

    # Compute the distance from data to their center
    dist_all = cdist(data, centers, metric='cityblock')
    dist = []

    for j in range(len(dist_all)):
        dist.append(dist_all[j,kmeans.labels_[j]])
    dist = pd.DataFrame(dist)

    # Combine into one table
    dist_pd = pd.concat([pd.DataFrame(data),dist,labels],axis = 1)
    dist_pd.columns = ["N","dist","cluster"]

    # Put the 'centers position' and 'max_dist' in to the table
    centers_pd = pd.DataFrame(centers)
    centers_pd.reset_index(drop=False,inplace=True)
    centers_pd.columns = ['cluster','center']

    max_dist = pd.DataFrame(dist_pd.groupby("cluster")['dist'].max())  # 每个cluster中dist的最大值
    max_dist.reset_index(drop=False,inplace=True)
    max_dist.columns = ['cluster','max_dist']

    # Combine
    centers_pd = pd.merge(centers_pd,max_dist,on='cluster')
    data_pd = pd.merge(dist_pd,centers_pd,on='cluster',how="left")

    # Redefine the 'dist'
    data_pd['new_dist'] = data_pd['max_dist'] - data_pd['dist']+1

    # Compute the dist sum
    new_sum_dist = pd.DataFrame(data_pd.groupby("cluster")['new_dist'].sum())
    new_sum_dist.reset_index(drop=False,inplace=True)
    new_sum_dist.columns = ['cluster','new_sum_dist']

    # Combine
    data_pd = pd.merge(data_pd,new_sum_dist,on='cluster',how="left")

    # Compute the prob
    data_pd['prob'] = data_pd['new_dist']/data_pd['new_sum_dist']

    # N_news = int(P*len(data))
    # Get the amount of data to be oversampled (in each cluster)
    cluster_size = np.array(data_pd.groupby("cluster").count().iloc[:,0])
    N_news_ls = np.array(cluster_size*P,dtype=int) # 根据cluster大小分配抽样规模

    ##################### Oversample #####################

    new_data_i = pd.DataFrame()       # 新生成的data‘N’

    for j in range(kmeans.n_clusters):
        # Get all data in cluster j

        data_j = data_pd[data_pd.loc[:,'cluster'] == j]
        data_j = data_j.reset_index(drop=True)

        data_j.sort_values(by='prob',ascending = False,inplace = True,ignore_index = True) # 降序排序,并且重新index一下。降序是为了下面抽取idx

        # Sample according to the prob
        assert kmeans.n_clusters == n_cluster,f"kmeans.n_cluster = {kmeans.n_clusters}!!"
        assert N_news_ls.__len__() == n_cluster,f"N_news_ls len = {N_news_ls.__len__()}!!"
        new_idx = random.choice(a = data_j.shape[0], p = data_j.prob,size=N_news_ls[j])
        new_data_j = data_j.loc[new_idx,'N'].reset_index(drop=True)

        new_data_i = pd.concat([new_data_i,pd.DataFrame(new_data_j)],axis=0)

    new_data_i.reset_index(drop=True, inplace=True)
    assert new_data_i.shape[0] == N_news_ls.sum(),"Did not oversample enough data"

    ##################### Redesign the data table (Compute the 'p' value)#####################

    # Concat the old with the new
    N_data = pd.concat([pd.DataFrame(data,columns=['N']),new_data_i],axis=0,ignore_index=True)
    # Count 'cnt' to compute P
    pcount = collections.Counter(N_data.iloc[:,0])
    tmp = pd.DataFrame.from_dict(pcount,orient='index').reset_index()
    tmp.columns = ['N','cnt']
    # Compute P
    tmp['P'] = tmp.cnt/ tmp.shape[0]
    N_data = pd.merge(N_data,tmp,how='left',on=['N'])
    # Repeat data records according to 'cnt'
    output_data = N_data.loc[N_data.index.repeat(N_data['cnt'])]

    ##################### 如果需要重新整理粒度，就不要drop'cnt'这一列
    if not RESCALE:
        output_data.drop(columns=['cnt'],inplace=True)
        output_path = os.path.join(target_path_DA,target_all_path[i])
        output_data.to_csv(output_path,header=True,index=False,encoding="utf-8")
    elif RESCALE:
        output_data.sort_values(by='N',ignore_index=True,inplace=True)
        # Drop duplicates
        data_i_df = output_data.drop_duplicates(inplace = False,ignore_index=True)

        # 现在数据是无重复版本的，'cnt'记录了这个record重复出现的次数
        # New added: Rearrange the data according to their length
        data_lenth = data_i_df.shape[0]         # Data length
        dele_idx = []                           # Idx to be deleted

        if(data_lenth >= 0):
            j = 0
            while (j < data_lenth):
                # N值-1恰好是SCALE倍
                if((data_i_df.iloc[j,0]-1)%SCALE == 0):
                    k = j+1    # 用k记录位置
                    # k不是data最后一个值 and [k]在+SCALE的范围内
                    while((k < data_lenth) and (data_i_df.iloc[j,0]+SCALE > data_i_df.iloc[k,0])):
                        # 更新[j]的prob值和cnt值：叠加
                        data_i_df.iloc[j,1] += data_i_df.iloc[k,1]
                        data_i_df.iloc[j,2] += data_i_df.iloc[k,2]
                        # 删除[k]
                        dele_idx.append(k)
                        k += 1
                    # j 从 k（其实是k+1）的位置继续
                    j = k
                # N值不是SCALE倍且未被并入任何一个已存在的开头
                else:
                    # 原地修改（减小）成一个新的区间的开头,
                    while((data_i_df.iloc[j,0]-1) %SCALE != 0):
                        data_i_df.iloc[j,0] -= 1
                    # 注意不需要j+1，下一次循环从当前开始，check后面的n需不需要并进来
                    # j += 1

        ######### 如果只想保存改动的data，把下面都拿到上面的if里面来
        # Save i_th training data file
        # save_i_trainfile(i)
        # Dele
        data_i_df = data_i_df.drop(dele_idx,axis = 0).copy()

        # Reconstruct and repeat data according to 'cnt_n_2'
        data_i_df = data_i_df.loc[data_i_df.index.repeat(data_i_df['cnt'])]
        data_i_df = data_i_df[col_names_2]

        # Get the output path
        output_path = os.path.join(target_path_DA,target_all_path[i])
        data_i_df.to_csv(output_path,header=True,index=False,encoding="utf-8")


print("Done")

100%|██████████| 1196/1196 [01:04<00:00, 18.59it/s]

Done


# 3. drop uniform auctions

In [101]:
target_ls = []
arr = []
K = 2

# target_path_DA = r"../data/targets_5"
for i in tqdm(range(len(target_all_path))):
    target_path_i_path = os.path.join(target_path_DA,target_all_path[i])
    target_df = pd.read_csv(target_path_i_path,encoding="utf-8")

    p = target_df.P.nunique()
    if p <= K:
        target_ls.append(i)
    else:
        arr.append(i)

print(f"target data近似为均匀分布的数量: {len(target_ls)}, 占比: {len(target_ls)/len(target_all_path)}")

100%|██████████| 1196/1196 [00:00<00:00, 1627.70it/s]

target data近似为均匀分布的数量: 16, 占比: 0.013377926421404682


In [102]:
# 保存idx for training

if RESCALE:
    arr_path_DA = r"arr_targets_"+str(SCALE)+"_DA_P="+str(P)+r"_N_c="+str(N_cluster)+r"_K="+str(K)
else:
    arr_path_DA = r"arr_targets_DA_P="+str(P)+r"_N_c="+str(N_cluster)+r"_K="+str(K)

arr_path = os.path.join(arr_path_root,arr_path_DA)
arr = np.array(arr)
np.save(arr_path,arr)